In [5]:
import pandas as pd
from word2number import w2n
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [6]:
def read_split_data(filename):
    file = open(filename, "r").read()
    list_rows = []
    file = file.split("\n")
    for row in range(0, len(file) - 1):
        dict_rows = {}
        splits = file[row].split(" ")
        for s in range(len(splits)):
            if s == 0:
                if int(splits[s]) == 1:
                    dict_rows[s] =  int(splits[0])
                else:
                    dict_rows[s] = -1
            else:
                index,val = [float(e) for e in splits[s].split(':')]
                dict_rows[index] = val
        list_rows.append(dict_rows)
    df =  pd.DataFrame.from_dict(list_rows)
    df = df.fillna(0)
    return df

In [7]:
df_glove_train = read_split_data('glove.train.libsvm')
df_glove_test = read_split_data('glove.test.libsvm')

In [8]:
dataset =  pd.read_csv('misc-attributes-train.csv')
eval =  pd.read_csv('misc-attributes-eval.csv')
test = pd.read_csv('misc-attributes-test.csv')

In [9]:
defendants_age = []
defendant_gender=[]
num_victims=[]
offence_category=[]
offence_subcategory=[]

In [10]:
def change_age(x):
    
    for i,age in enumerate(x['defendant_age']):
#         print("age",age)
        # print(i)
        age = age.strip("(  ) -")
        if age != "not known":
            age = age.replace("years","")
            age = age.replace("about", "")
            age = age.replace("age","")
            age = age.replace("of", "" )
            age = age.replace("old", "")
            age = age.strip()
            if age.find(" ") >= 0:
                temp = age.split(" ")
                # print(temp)
                age = '-'.join(temp)
            syns = wordnet.synsets(age.strip())
            # print("A",age)
            # print("S",syns[0].lemmas()[0].name())
            age = syns[0].lemmas()[0].name()
            if age.find("-") >= 0:
                temp = age.split("-")
                # print(temp)
                age = ' '.join(temp)
            # print(age.strip())
            defendants_age.append(w2n.word_to_num(age.strip()))
        else:
            defendants_age.append(int(0))

In [11]:
change_age(dataset)

In [12]:
gender = ['female', 'indeterminate', 'male']
off_cat = ['breakingPeace','damage','deception','kill','miscellaneous','royalOffences','sexual','theft','violentTheft']
off_sub_cat = ['animalTheft','arson','assault','assaultWithIntent','assaultWithSodomiticalIntent','bankrupcy','bigamy','burglary','coiningOffences',
 'concealingABirth', 'conspiracy', 'embezzlement','extortion','forgery','fraud','gameLawOffence','grandLarceny','highwayRobbery',
 'housebreaking','illegalAbortion','indecentAssault','infanticide','keepingABrothel','kidnapping','libel','mail','manslaughter',
 'murder','other','perjury','pervertingJustice','pettyLarceny','pettyTreason','piracy','pocketpicking','rape','receiving',
 'religiousOffences','returnFromTransportation','riot','robbery','seditiousLibel','seditiousWords','seducingAllegiance',
 'shoplifting','simpleLarceny','sodomy','stealingFromMaster','taxOffences','theftFromPlace','threateningBehaviour','treason',
 'wounding','coiningOffences','vagabond']

In [13]:
def factorize_gender(x):
    for i,g in enumerate(x['defendant_gender']):
        if g in gender:
            defendant_gender.append(gender.index(g))
#     return defendant_gender
            

In [14]:
def factorize_offence_category(x):
    for i,off in enumerate(x['offence_category']):
        if off in off_cat:
            offence_category.append(off_cat.index(off))
#     return offence_category

            
    

In [15]:
def factorize_offence_subcategory(x):
    for i,off_sub in enumerate(x['offence_subcategory']):
        if off_sub in off_sub_cat:
            offence_subcategory.append(off_sub_cat.index(off_sub))
#     return offence_subcategory
            

In [16]:
num_victims = list(dataset['num_victims'])

In [17]:
factorize_gender(dataset)
factorize_offence_category(dataset)
factorize_offence_subcategory(dataset)

In [18]:
df = pd.DataFrame(list(zip(defendants_age,defendant_gender,num_victims,offence_category,offence_subcategory)),  columns =["defendants_age" ,"defendants_gender","num_victims","offence_category","offence_sub_category"])


In [19]:
df

,defendants_age,defendants_gender,num_victims,offence_category,offence_sub_category
0,62,0,1,7,49
1,17,2,1,7,34
2,0,2,1,7,34
3,0,2,1,7,45
4,52,2,1,7,34
...,...,...,...,...,...
17495,0,2,1,7,49
17496,0,2,0,7,0
17497,0,2,1,7,18
17498,0,2,0,4,28


In [20]:
train_numpy = df.iloc[:,:].values

In [21]:
train_label = df_glove_train.iloc[:,0].values

In [22]:
sc = StandardScaler()

In [23]:
X_train = sc.fit_transform(train_numpy)

In [24]:
regressor = RandomForestClassifier(n_estimators=300,random_state=0,max_depth=7)

In [25]:
regressor.fit(X_train,train_label)

RandomForestClassifier(max_depth=7, n_estimators=300, random_state=0)

In [26]:
y_pred = regressor.predict(X_train)
accuracy_score(train_label,y_pred)

0.7935428571428571

In [27]:
##### for testing ######
defendants_age = []
defendant_gender=[]
num_victims=[]
offence_category=[]
offence_subcategory=[]

In [28]:
change_age(test)
factorize_gender(test)
factorize_offence_category(test)
factorize_offence_subcategory(test)

In [29]:
num_victims = list(test['num_victims'])

In [30]:
df_test = pd.DataFrame(list(zip(defendants_age,defendant_gender,num_victims,offence_category,offence_subcategory)),  columns =["defendants_age" ,"defendants_gender","num_victims","offence_category","offence_sub_category"])


In [31]:
test_numpy = df_test.iloc[:,:].values

In [32]:
test_label = df_glove_test.iloc[:,0].values

In [33]:
X_test = sc.fit_transform(test_numpy)

In [34]:
y_pred_test = regressor.predict(X_test)
accuracy_score(test_label,y_pred_test)

0.7964444444444444

In [35]:
###### for eval #######
defendants_age = []
defendant_gender=[]
num_victims=[]
offence_category=[]
offence_subcategory=[]

In [36]:

factorize_gender(eval)
factorize_offence_category(eval)
factorize_offence_subcategory(eval)
num_victims = list(eval['num_victims'])

In [37]:
for i,age in enumerate(eval['defendant_age']):
#         print("age",age)
        # print(i)
        age = age.strip("(  ) -")
        if age != "not known":
            age = age.replace("years","")
            age = age.replace("about", "")
            age = age.replace("age","")
            age = age.replace("of", "" )
            age = age.replace("old", "")
            age = age.replace("Year", "")
            age = age.replace("his", "")
            age = age.replace("Age", "")
            age = age.strip()
            age = age.strip("d")
            
            if age.find(" ") >= 0:
                temp = age.split(" ")
                if "and" in temp or "or" in temp:
                    age = temp[0]
#                     temp.remove("and")
#                 if "months" in temp or "month" in temp:
#                     temp.remove("months")
# #                     temp.remove("month")
                # print(temp)
                else:
                    age = '-'.join(temp)
#             print("**",age)
            syns = wordnet.synsets(age.strip())
#             print("syns",syns)
            # print("A",age)
            # print("S",syns[0].lemmas()[0].name())
            age = syns[0].lemmas()[0].name()
            if age.find("-") >= 0:
                temp = age.split("-")
                # print(temp)
                age = ' '.join(temp)
#             print(age)
#             defendants_age.append(w2n.word_to_num(age.strip()))
            try:
                defendants_age.append(w2n.word_to_num(age.strip()))
            except:
                defendants_age.append(int(0))
        else:
            defendants_age.append(int(0))

In [38]:
df_eval = pd.DataFrame(list(zip(defendants_age,defendant_gender,num_victims,offence_category,offence_subcategory)),  columns =["defendants_age" ,"defendants_gender","num_victims","offence_category","offence_sub_category"])


In [39]:
eval_numpy = df_eval.iloc[:,:].values

In [40]:
eval_label = eval.iloc[:,0].values

In [41]:
X_eval = sc.fit_transform(eval_numpy)

In [42]:
y_pred_eval = regressor.predict(X_eval)

In [43]:
len(offence_subcategory)

5250

In [44]:
import csv

In [45]:
with open('random_forest_final.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["example_id", "label"])
    for i in range(len(y_pred_eval)):
        if y_pred_eval[i]== -1:
            val = 0
        else:
            val = 1
        writer.writerow([i, val])